# Assignment week 6

In [1]:
import pandas as pd
import seaborn as sns
import sklearn as sk
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 

## Data pre-processing steps

In [4]:
reviews = pd.read_csv('data-clothing-reviews.csv')
reviews.head()

,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
0,0,767,33,NaN,Absolutely wonderful - silky and sexy and comf...,4,1,0,Initmates,Intimate,Intimates
1,1,1080,34,NaN,Love this dress! it's sooo pretty. i happene...,5,1,4,General,Dresses,Dresses
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
3,3,1049,50,My favorite buy!,"I love, love, love this jumpsuit. it's fun, fl...",5,1,0,General Petite,Bottoms,Pants
4,4,847,47,Flattering shirt,This shirt is very flattering to all due to th...,5,1,6,General,Tops,Blouses


In [23]:
df_subset = reviews[(reviews["Class Name"] == "Dresses")]

df_subset.dropna(axis=0,how='any',thresh=None,subset=None,inplace=True)
df_subset.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Unnamed: 0,Clothing ID,Age,Title,Review Text,Rating,Recommended IND,Positive Feedback Count,Division Name,Department Name,Class Name
2,2,1077,60,Some major design flaws,I had such high hopes for this dress and reall...,3,0,0,General,Dresses,Dresses
5,5,1080,49,Not for the very petite,"I love tracy reese dresses, but this one is no...",2,0,4,General,Dresses,Dresses
8,8,1077,24,Flattering,I love this dress. i usually get an xs but it ...,5,1,0,General,Dresses,Dresses
9,9,1077,34,Such a fun dress!,"I'm 5""5' and 125 lbs. i ordered the s petite t...",5,1,0,General,Dresses,Dresses
10,10,1077,53,Dress looks like it's made of cheap material,Dress runs small esp where the zipper area run...,3,0,14,General,Dresses,Dresses


## Text pre-processing steps resulting in a document-feature matrix

In [30]:
def positive_or_negative(x):
    if(x > 3): 
        return 'positive'
    else: 
        return 'negative'

df_subset['Rate'] = df_subset['Rating'].apply(positive_or_negative)

df_subset['Rate'].value_counts()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


positive    4045
negative    1326
Name: Rate, dtype: int64

In [33]:
from sklearn.feature_extraction.text import CountVectorizer 

text = df_subset['Review Text'].values.astype('U') 
vect = CountVectorizer(stop_words='english') 
vect = vect.fit(text) 
vect
feature_names = vect.get_feature_names() 

print(f"There are {len(feature_names)} words in the vocabulary. A selection: {feature_names[500:520]}")

There are 7747 words in the vocabulary. A selection: ['allusion', 'allusione', 'almsot', 'alr', 'alright', 'als', 'altar', 'alter', 'alteration', 'alterations', 'altered', 'altering', 'alternate', 'alternations', 'alternative', 'althetic', 'altho', 'altogether', 'am5', 'amadi']


In [35]:
docu_feat = vect.transform(text) #The transform method from the CountVectorizer object creates the matrix
print(docu_feat[0:500,0:500]) #Let's print a little part of the matrix: the first 50 words & documents

  (1, 7)	1
  (3, 71)	1
  (3, 214)	1
  (12, 480)	1
  (13, 200)	1
  (14, 78)	1
  (14, 408)	1
  (14, 420)	1
  (15, 37)	1
  (16, 3)	1
  (16, 44)	1
  (17, 11)	1
  (17, 236)	1
  (18, 54)	1
  (19, 39)	1
  (21, 59)	1
  (21, 216)	1
  (21, 225)	1
  (22, 229)	1
  (23, 102)	1
  (23, 178)	1
  (23, 222)	1
  (23, 243)	1
  (24, 403)	1
  (26, 11)	2
  :	:
  (475, 96)	1
  (475, 455)	2
  (476, 63)	1
  (476, 200)	1
  (476, 334)	1
  (477, 344)	1
  (478, 156)	1
  (479, 216)	1
  (479, 368)	1
  (482, 309)	2
  (484, 130)	1
  (489, 362)	1
  (490, 112)	1
  (492, 11)	2
  (492, 364)	1
  (493, 50)	1
  (493, 480)	1
  (497, 442)	1
  (498, 334)	1
  (499, 11)	1
  (499, 165)	1
  (499, 187)	1
  (499, 219)	1
  (499, 248)	1
  (499, 451)	1


## Split the file into a training and a test set - Naive Bayes

In [39]:
from sklearn.naive_bayes import MultinomialNB

nb = MultinomialNB()
X = docu_feat
y = df_subset['Rate'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

nb = nb.fit(X_train, y_train)
y_test_p = nb.predict(X_test)

nb.score(X_test, y_test)

0.8542183622828784

## Evaluate the performance of the model 

In [40]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_test_p)
cm

array([[ 238,  168],
       [  67, 1139]])

In [41]:
nb.classes_

array(['negative', 'positive'], dtype='<U8')

In [44]:
conf_matrix = pd.DataFrame(cm, index = ["Negative (actual)", "Positive (actual)"], columns = ["Negative (predicted)", "Positive (predicted)"])
conf_matrix

,Negative (predicted),Positive (predicted)
Negative (actual),238,168
Positive (actual),67,1139


Calculating the different variables: 

$accuracy = \frac{238 + 1139}{238 + 67 + 168 + 1139} = 0.85$

The accuracy is exactly the same as the knn-score


How much of the predicted Negative is actually Positive? 

$recall = \frac{67}{67 + 1139} = 0.06$


How much of the predicted Positive is actually Negative?

$precision = \frac{168}{238 + 168} = 0.41$

In [47]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_test_p))

              precision    recall  f1-score   support

    negative       0.78      0.59      0.67       406
    positive       0.87      0.94      0.91      1206

    accuracy                           0.85      1612
   macro avg       0.83      0.77      0.79      1612
weighted avg       0.85      0.85      0.85      1612

